# Classical Monte Carlo of $S_N$ Model on Hypertree Lattice

In [1]:
%reload_ext snakeviz
%run 'MC.py'

## Hypertree Model

### Model Setup
* Lattice: hypertree (branching MERA like, binary branching and binary merging)
* Degrees of freedom on each site is a permutation group element $g_i\in S_N$.
* Hamiltonian:
$$H = -\sum_{\langle ij\rangle} K_{ij}\chi(g_i,g_j),$$
where $\chi(g_i,g_j)=\text{Tr}g_i^{-1}g_j$ is the group character (the number of cycles).

In [2]:
HypertreeLattice(4)

[<2,0,0>, <2,1,0>, <2,2,0>, <2,3,0>, <1,0,0>, <1,0,1>, <1,1,0>, <1,1,1>, <0,0,0>, <0,0,1>, <0,0,2>, <0,0,3>]

In [3]:
Group(3).chi

[[0, -1, -1, -2, -2, -1],
 [-1, 0, -2, -1, -1, -2],
 [-1, -2, 0, -1, -1, -2],
 [-2, -1, -1, 0, -2, -1],
 [-2, -1, -1, -2, 0, -1],
 [-1, -2, -2, -1, -1, 0]]

### Example
Generate model, run, get hist and energy, make measurements.
* use **block Gibbs sampling** + **heatbath update**, alternatively sampling on A and B sublattices.
 * A sublattice is the one that contains the most UV layer
 * B sublattice contains the rest
 * the boundary is labeled by C
* hist and magnetization only conts the bulk spins, boundary spins are not taken into account.

In [2]:
sys = LatticeModel(HypertreeLattice(4,[1.,0.2]),Group(3)).run(100)
for k in range(10):
    sys.run(mode = 1)
    print(sys.config,sys.hist,sys.energy)
sys.measure(1000,monitor=[0,1,2,3])

[0 2 0 1 0 0 5 1 0 0 0 0] [4 2 1 0 0 1] 5.199999999999999
[0 3 3 3 0 5 1 3 0 0 0 0] [2 1 0 4 0 1] 7.599999999999999
[4 1 3 1 1 0 2 1 0 0 0 0] [1 4 1 1 1 0] 9.999999999999998
[0 2 0 3 0 0 2 0 0 0 0 0] [5 0 2 1 0 0] 3.9999999999999987
[2 0 0 3 0 5 2 2 0 0 0 0] [3 0 3 1 0 1] 9.999999999999996
[0 5 2 2 0 0 3 2 0 0 0 0] [3 0 3 1 0 1] 4.399999999999997
[0 3 2 3 0 0 2 2 0 0 0 0] [3 0 3 2 0 0] 3.5999999999999974
[1 2 2 4 0 0 0 0 0 0 0 0] [4 1 2 0 1 0] 6.799999999999999
[1 2 0 1 4 0 0 0 0 0 0 0] [4 2 1 0 1 0] 8.399999999999999
[0 5 0 5 0 1 5 0 0 0 0 0] [4 1 0 0 0 3] 6.799999999999997


{'energy1': 6.492800000000001,
 'energy2': 47.164479999999934,
 'magnet1': array([ 0.416   ,  0.12875 ,  0.141375,  0.08725 ,  0.0875  ,  0.139125]),
 'magnet2': array([[ 0.20721875,  0.04709375,  0.05221875,  0.03079688,  0.0299375 ,
          0.04873438],
        [ 0.04709375,  0.0350625 ,  0.01214063,  0.00970312,  0.009875  ,
          0.014875  ],
        [ 0.05221875,  0.01214063,  0.04076562,  0.01082812,  0.01059375,
          0.01482812],
        [ 0.03079688,  0.00970312,  0.01082812,  0.01996875,  0.0056875 ,
          0.01026563],
        [ 0.0299375 ,  0.009875  ,  0.01059375,  0.0056875 ,  0.02109375,
          0.0103125 ],
        [ 0.04873438,  0.014875  ,  0.01482812,  0.01026563,  0.0103125 ,
          0.04010938]]),
 'spins': array([[ 0.578,  0.252,  0.234,  0.166],
        [ 0.092,  0.162,  0.152,  0.172],
        [ 0.124,  0.167,  0.179,  0.172],
        [ 0.033,  0.127,  0.14 ,  0.162],
        [ 0.037,  0.118,  0.136,  0.161],
        [ 0.136,  0.174,  0.159,  0.

The CPU intensive MC samplings are performed by the FORTRAN extension MC. The module MC contains two parts:
* core (for MC sampling and update macroscopic observibles)
* physics (for data collection and measurements)

In [3]:
import MC
print(MC.__doc__)

This module 'MC' is auto-generated with f2py (version:2).
Functions:
Fortran 90/95 modules:
  core --- chi,jlst,klst,nsite,na,dof,nlst,config,nc,irng,energy,nb,hist,init(),run(),get_energy(),get_hist(),dump(),load()  physics --- nspin,spins,energy2,magnet1,monitor,energy1,magnet2,measure().


### Profiling

In [4]:
%snakeviz LatticeModel(HypertreeLattice(4,[1.,0.2]),Group(3)).run(1000)

 
*** Profile stats marshalled to file '/var/folders/lt/kgph6qbj6y306pfhbyd1kn2r0000gn/T/tmp3lqyahqv'. 
